In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_32x8x8_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_32x8x8_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()

In [2]:
model = keras.Sequential([
    layers.Input(shape=(32, 8, 8, 3)),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(4, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(2, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.AvgPool3D((1, 2, 2)),
    layers.Conv3D(1, 1, 1),
    layers.Flatten(),
], name='NoobHeart')
model.compile(optimizer='adam', loss='mae')
print(f'Flops per frame:{get_flops(model, input_sig=[tf.TensorSpec([1, 32, 8, 8, 3])])/32:.0f}')
model.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:5790
Model: "NoobHeart"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization (LayerNo (None, 32, 8, 8, 3)       64        
_________________________________________________________________
conv3d (Conv3D)              (None, 32, 4, 4, 4)       100       
_________________________________________________________________
layer_normalization_1 (Layer (None, 32, 4, 4, 4)       64        
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 32, 2, 2, 2)       66        
_________________________________________________________________
layer_normalization_2 (Layer (None, 32, 2, 2, 2)       64        
_________________________________________________________________
average_pooling3d (AveragePo (None, 32, 1, 1, 2)       0         
_____________

In [4]:
model.fit(train.shuffle(2000).batch(128), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/NoobHeart_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
model.load_weights('../weights/NoobHeart_CCNU.h5')

Epoch 1/10
1189/1189 [==============================] - 44s 36ms/step - loss: 0.7653 - val_loss: 0.7378

Epoch 00001: val_loss improved from inf to 0.73776, saving model to ../weights\NoobHeart_CCNU.h5
Epoch 2/10
1189/1189 [==============================] - 6s 5ms/step - loss: 0.7349 - val_loss: 0.7304

Epoch 00002: val_loss improved from 0.73776 to 0.73039, saving model to ../weights\NoobHeart_CCNU.h5
Epoch 3/10
1189/1189 [==============================] - 6s 5ms/step - loss: 0.7268 - val_loss: 0.7081

Epoch 00003: val_loss improved from 0.73039 to 0.70806, saving model to ../weights\NoobHeart_CCNU.h5
Epoch 4/10
1189/1189 [==============================] - 6s 5ms/step - loss: 0.7194 - val_loss: 0.6997

Epoch 00004: val_loss improved from 0.70806 to 0.69967, saving model to ../weights\NoobHeart_CCNU.h5
Epoch 5/10
1189/1189 [==============================] - 6s 5ms/step - loss: 0.7181 - val_loss: 0.6994

Epoch 00005: val_loss improved from 0.69967 to 0.69941, saving model to ../weights\

In [3]:
model.load_weights('../weights/NoobHeart.h5')

In [5]:
eval_on_dataset(test_set_CCNU, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_CCNU.h5')
get_metrics('../results/NoobHeart_CCNU_CCNU.h5',)
#get_metrics_HRV('../results/NoobHeart_CCNU_CCNU.h5',)

100%|██████████| 179/179 [00:16<00:00, 10.59it/s]


{'Sliding window': {'MAE': 1.688, 'RMSE': 5.53, 'R': 0.84964},
 'Whole video': {'MAE': 0.838, 'RMSE': 2.76, 'R': 0.9577}}

In [6]:
eval_on_dataset(test_set_CCNU_rPPG, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_CCNU_rPPG.h5')
get_metrics('../results/NoobHeart_CCNU_CCNU_rPPG.h5',)
#get_metrics_HRV('../results/NoobHeart_CCNU_CCNU_rPPG.h5',)

100%|██████████| 54/54 [00:03<00:00, 16.43it/s]


{'Sliding window': {'MAE': 1.511, 'RMSE': 4.403, 'R': 0.89753},
 'Whole video': {'MAE': 0.784, 'RMSE': 1.463, 'R': 0.9881}}

In [7]:
eval_on_dataset(test_set_PURE, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_PURE.h5')
get_metrics('../results/NoobHeart_CCNU_PURE.h5',)
#get_metrics_HRV('../results/NoobHeart_CCNU_PURE.h5',)

100%|██████████| 59/59 [00:02<00:00, 22.77it/s]


{'Sliding window': {'MAE': 1.121, 'RMSE': 6.905, 'R': 0.95399},
 'Whole video': {'MAE': 0.438, 'RMSE': 0.701, 'R': 0.99968}}

In [8]:
eval_on_dataset(test_set_UBFC_rPPG2, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_CCNU_UBFC.h5')
get_metrics('../results/NoobHeart_CCNU_UBFC.h5',)
#get_metrics_HRV('../results/NoobHeart_CCNU_UBFC.h5',)

100%|██████████| 42/42 [00:01<00:00, 26.05it/s]


{'Sliding window': {'MAE': 1.762, 'RMSE': 4.327, 'R': 0.96993},
 'Whole video': {'MAE': 1.155, 'RMSE': 1.695, 'R': 0.99633}}

In [11]:
eval_on_dataset(test_set_MMPD, model, 32, (8, 8), step=0.5, batch=32, save='../results/NoobHeart_CCNU_MMPD.h5')
get_metrics('../results/NoobHeart_CCNU_MMPD.h5', motion='Stationary', skin_color='3', light=['LED-high', 'Incandescent', 'LED-low'])

100%|██████████| 660/660 [00:35<00:00, 18.64it/s]


{'Sliding window': {'MAE': 4.962, 'RMSE': 9.607, 'R': 0.52993},
 'Whole video': {'MAE': 2.776, 'RMSE': 6.311, 'R': 0.76314}}